In [1]:
import pandas as pd
import os

Procedimiento para dividir el archivo de 5Gb en 20 partes para su carga incremental

In [ ]:
def dividir_json_grande(archivo_entrada, num_partes):
    # Contar el número total de líneas/objetos en el archivo JSON
    with open(archivo_entrada, 'r', encoding='utf-8') as archivo:
        total_lineas = sum(1 for _ in archivo)
    
    # Calcular el tamaño de cada parte
    tamaño_parte = total_lineas // num_partes
    if total_lineas % num_partes != 0:
        tamaño_parte += 1

    # Leer y dividir el archivo en partes
    with open(archivo_entrada, 'r', encoding='utf-8') as archivo:
        for i in range(num_partes):
            archivo_salida = f'parte_{i + 1}.json'
            with open(archivo_salida, 'w', encoding='utf-8') as salida:
                salida.write('[')  # Comenzar un array JSON
                for j in range(tamaño_parte):
                    linea = archivo.readline()
                    if not linea:
                        break
                    salida.write(linea.rstrip())
                    if j < tamaño_parte - 1 and not archivo.tell() == total_lineas:
                        salida.write(',')
                salida.write(']')  # Terminar el array JSON
            # Si el archivo original ha terminado, salir del bucle
            if not linea:
                break

# Usar la función para dividir el archivo 'archivo.json' en 10 partes
dividir_json_grande('D:/PROYECTO FINAL/Yelp/review.json', 20)

In [2]:
#df_reviews_yelp = pd.read_json('D:/PROYECTO FINAL/Yelp/review/parte_16.json')
df_reviews_yelp = pd.read_json('C:/Data/Yelp/review/parte_1.json')
df_reviews_yelp.head(3)

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30


In [3]:
df_reviews_yelp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 466019 entries, 0 to 466018
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   review_id    466019 non-null  object        
 1   user_id      466019 non-null  object        
 2   business_id  466019 non-null  object        
 3   stars        466019 non-null  int64         
 4   useful       466019 non-null  int64         
 5   funny        466019 non-null  int64         
 6   cool         466019 non-null  int64         
 7   text         466019 non-null  object        
 8   date         466019 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(4), object(4)
memory usage: 32.0+ MB


In [4]:
#df_estados_USA= pd.read_csv("D:/PROYECTO FINAL/Google Maps/estados_usa.csv",delimiter = ';', encoding = "utf-8")
df_estados_USA= pd.read_csv("C:/Data/GMaps/estados_usa.csv",delimiter = ';', encoding = "utf-8")
df_estados_USA= df_estados_USA.rename(columns={'nombre_largo': 'estado', 'nombre_corto':'state'}) # cambiar nombre de la columna
df_estados_USA= df_estados_USA.drop(['codigos'], axis=1) 
df_estados_USA['estado']= df_estados_USA['estado'].convert_dtypes(convert_string=True)
df_estados_USA['estado']= df_estados_USA['estado'].str.strip()  # quita los espacios vacios

In [5]:
#df_business = pd.read_pickle("D:/PROYECTO FINAL/Yelp/business.pkl")
df_business = pd.read_pickle("C:/Data/Yelp/business.pkl")
# Se cargan los business para filtrar los reviews de la categoria restaurantes solamente
df_business = df_business.loc[:, ~df_business.columns.duplicated()]
df_business.drop_duplicates('business_id',inplace=True)
df_business['categories'] = df_business['categories'].str.split(',')
df_business= df_business.dropna(subset=['categories']) # Elimina datos nulos de la columna
df_business['Es_Restaurant'] = df_business['categories'].apply(lambda x: 'Restaurants' in x)
df_business= df_business[df_business['Es_Restaurant']]
df_business= df_business.dropna(subset=['state']) # Elimina datos nulos de la columna
df_business['state']= df_business['state'].str.strip()  # quita los espacios vacios

In [6]:
df_business = df_business.merge(df_estados_USA, on='state', how='left')

In [9]:
df_business.head(3)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,Es_Restaurant,estado
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,CA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","[Restaurants, Food, Bubble Tea, Coffee & Te...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",True,California
1,9OG5YkX1g2GReZM0AskizA,Romano's Macaroni Grill,5505 S Virginia St,Reno,AB,89502,39.476117,-119.789339,2.5,339,1,"{'RestaurantsGoodForGroups': 'True', 'Restaura...","[Restaurants, Italian]","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'...",True,<NA>
2,tMkwHmWFUEXrC9ZduonpTg,The Green Pheasant,215 1st Ave S,Nashville,TN,37201,36.159886,-86.773197,4.0,161,0,"{'RestaurantsGoodForGroups': 'True', 'HappyHou...","[Restaurants, Japanese, Seafood]","{'Wednesday': '16:0-22:0', 'Thursday': '16:0-2...",True,Tennessee


In [10]:
lista_estados= ['Florida', 'Pennsylvania', 'Tennessee', 'California', 'Texas', 'New York']
df_business= df_business[df_business['estado'].isin(lista_estados)]

In [11]:
df_business.shape

(7854, 16)

In [13]:
df_business= df_business[['business_id', 'name']]

In [14]:
df_business.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7854 entries, 0 to 15289
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   business_id  7854 non-null   object
 1   name         7854 non-null   object
dtypes: object(2)
memory usage: 184.1+ KB


In [15]:
df_reviews_yelp = df_reviews_yelp.merge(df_business, on='business_id', how='inner')

In [16]:
df_reviews_yelp = df_reviews_yelp[(df_reviews_yelp['date'].dt.year >= 2010) & (df_reviews_yelp['date'].dt.year <= 2021)]

In [17]:
df_reviews_yelp.head(3)

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,name
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11,Turning Point of North Wales
1,UBp0zWyH60Hmw6Fsasei7w,4Uh27DgGzsp6PqrH913giQ,otQS34_MymijPTdNBoBdCw,4,0,2,0,The bun makes the Sonoran Dog. It's like a snu...,2011-10-27 17:12:05,BK Tacos
2,lUUhg8ltDsUZ9h0xnwY4Dg,RreNy--tOmXMl1en0wiBOg,cPepkJeRMtHapc_b2Oe_dw,4,1,0,1,I was really between 3 and 4 stars for this on...,2018-07-17 03:30:07,Naked Tchopstix Express


In [18]:
df_reviews_yelp= df_reviews_yelp.drop(['review_id','name'], axis=1)       #Elimina la columnas
df_reviews_yelp['platform']= 2
df_reviews_yelp= df_reviews_yelp[['business_id','user_id', 'date', 'stars','useful','funny','cool', 'text', 'platform']]

In [19]:
df_reviews_yelp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 36111 entries, 0 to 37150
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   business_id  36111 non-null  object        
 1   user_id      36111 non-null  object        
 2   date         36111 non-null  datetime64[ns]
 3   stars        36111 non-null  int64         
 4   useful       36111 non-null  int64         
 5   funny        36111 non-null  int64         
 6   cool         36111 non-null  int64         
 7   text         36111 non-null  object        
 8   platform     36111 non-null  int64         
dtypes: datetime64[ns](1), int64(5), object(3)
memory usage: 2.8+ MB


In [15]:
df_reviews_yelp.head(3)

,business_id,user_id,date,stars,useful,funny,cool,text,platform
0,DOoJrrsqGzZN2FqzvllKnQ,z8UXeMDV-2cJOddzAcHDFg,2019-05-12 00:25:30,5,0,0,0,"Fantastic food! Wish they were in Fairfax, Va...",2
1,DOoJrrsqGzZN2FqzvllKnQ,cs7C4VJ5ghS3CRrNBSgeMg,2019-07-28 14:00:31,4,0,0,0,I've eaten here 5 times or so now and it never...,2
2,DOoJrrsqGzZN2FqzvllKnQ,T62QEApuG0AvdeZhVLtL2w,2015-06-20 17:29:14,5,0,0,0,Nom Nom Nom!\n\nI love this place . Great marg...,2
